In [1]:
%%capture
%run 00_Dataset.ipynb

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
GRB = pd.read_csv(os.path.join('../data/','GRB','GRB_data_Wang_et_al_2011.csv'))
GRB = GRB.sort_values('z', ascending=False)
zGRB = GRB['z'].to_numpy()


In [4]:
if modelname == 'gaussianprocess':
    
    %run 02_Model_GuassianProcess.ipynb
    
    mu, mu_err = gpr.predict(zGRB.reshape(-1,1), return_std=True)

In [5]:
if modelname == 'lstm':
    
    %run 03_Model_RNN_BNN.ipynb
    
    model_u = model_uncertainity()
    model_u.compile(
        optimizer = tf.keras.optimizers.Adam(1e-3), 
        loss = tf.keras.losses.MeanSquaredError(), 
        metrics = tf.keras.metrics.RootMeanSquaredError()
    )
    model_u.load_weights(os.path.join(out_dir,'cp.ckpt'))
    
    zGRB_reconstruct = np.expand_dims(strided_app(zGRB, 4, 1), axis=-1)
    zGRB_reconstruct = np.append(np.array([zGRB[0], zGRB[0], zGRB[0], zGRB[0]]).reshape(1,-1,1), zGRB_reconstruct, axis=0)
    zGRB_reconstruct = np.append(np.array([zGRB[0], zGRB[0], zGRB[0], zGRB[1]]).reshape(1,-1,1), zGRB_reconstruct, axis=0)
    zGRB_reconstruct = np.append(np.array([zGRB[0], zGRB[0], zGRB[1], zGRB[2]]).reshape(1,-1,1), zGRB_reconstruct, axis=0)

    mu_reconstruct_uncertainity = []
    n = 1000
    for i in range(n):
        y_pred = model_u.predict(zGRB_reconstruct, batch_size=500)[:,-1].flatten()
        y_pred = mu_scaler.inverse_transform(y_pred.reshape(-1,1)).reshape(-1)
        mu_reconstruct_uncertainity.append(y_pred)

    mu_reconstruct_uncertainity = np.array(mu_reconstruct_uncertainity)

    # save output in a file
    mu = np.mean(mu_reconstruct_uncertainity, axis=0)
    mu_err = np.std(mu_reconstruct_uncertainity, axis=0)

2022-05-23 11:19:55.102628: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-05-23 11:19:55.102664: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: spk
2022-05-23 11:19:55.102669: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: spk
2022-05-23 11:19:55.102765: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-05-23 11:19:55.102782: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-05-23 11:19:55.102786: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.103.1 does not match DSO version 470.129.6 -- cannot find working devices in this configuration
2022-05-23 11:19:55.102998: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binar

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 4, 100)            40800     
                                                                 
 dropout (Dropout)           (None, 4, 100)            0         
                                                                 
 lstm_1 (LSTM)               (None, 4, 100)            80400     
                                                                 
 dropout_1 (Dropout)         (None, 4, 100)            0         
                                                                 
 dense (Dense)               (None, 4, 1)              101       
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable

In [6]:
np.savetxt(os.path.join(out_dir,"reconstruction.csv"), np.array([mu, mu_err]), delimiter=",")